## Load Data

In [2]:
# path_to_data = '/hpi/fs00/share/MPSS2021BA1/data/'
path_to_data = '../data/arima_results/'

In [ ]:
# 01 - ARIMA - Steady Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_12_o2_first1000 = pickle.load(input_file)
input_file.close()

In [ ]:
# 02 - ARIMA - Expanding Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_expanding_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_expanding_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_expanding_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_expanding_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arima_expanding_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arima_expanding_12_o2_first1000 = pickle.load(input_file)
input_file.close()

In [ ]:
# 03 - ARIMAX - Steady Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_12_o2_first1000 = pickle.load(input_file)
input_file.close()

In [ ]:
# 04 - ARIMAX - Expanding Train Size 12
import pickle
# Heart Rate - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_expanding_12_hr_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_expanding_12_hr_first1000 = pickle.load(input_file)
input_file.close()
# Blood Pressure - First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_expanding_12_bp_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_expanding_12_bp_first1000 = pickle.load(input_file)
input_file.close()
# O2 Saturation -  First 1000
input_file = open(path_to_data+'accuracy_dict_for_chunk_iterations_arimax_expanding_12_o2_first1000.pickle', 'rb')
accuracy_dict_for_chunk_iterations_arimax_expanding_12_o2_first1000 = pickle.load(input_file)
input_file.close()

## Prepare Data

In [ ]:
import pandas as pd
import time

# Transform nested dictionary into data frame using pd.concat()
def dict_to_df(selected_dict):

    starttime = time.time()
    print('START transforming dictionary.')
    chunkno = 0

    selected_dict_as_df = pd.DataFrame({})
    confusion_matrices_per_chunk_in_selected_dict = {}
    for chunkid in selected_dict:
        confusion_matrices_per_chunk_in_selected_dict[chunkid] = pd.concat(selected_dict[chunkid], axis=0).reset_index().rename(columns={'level_0':'ITERATION', 'level_1':'ACCURACY_TYPE'})

        chunkno = chunkno+1
        # runningtime = round(((time.time() - starttime) / 60), 5)
        # print('Completed chunk '+str(chunkid)+', running time in minutes: '+str(runningtime))

    selected_dict_as_df = pd.concat(confusion_matrices_per_chunk_in_selected_dict, axis=0).reset_index(level=0).rename(columns={'level_0':'CHUNK_ID_FILLED_TH'})

    endtime = round(((time.time() - starttime) / 60), 5)
    print('DONE transforming dictionary.')
    print('Completed '+str(chunkno)+' chunks in '+str(endtime)+' minutes')
    print('Number of chunk-iteration combinations:',sum(selected_dict_as_df.groupby(['CHUNK_ID_FILLED_TH'])['ITERATION'].nunique()))
    print('--------------------')

    return selected_dict_as_df

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up
def aggregate_to_confusion_matrix_per_chunk(df):
    confusion_matrix_per_chunk = pd.DataFrame({})
    confusion_matrix_per_chunk = df.groupby(['CHUNK_ID_FILLED_TH', 'ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum').reset_index()
    return confusion_matrix_per_chunk

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up
def aggregate_to_confusion_matrix_per_model(df):
    confusion_matrix_per_model = df.groupby(['ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum')
    return confusion_matrix_per_model

# Calculate statistics about the number of chunks as well as chunk iteration combinations 
def calc_stats(df):
    chunks = len(df.CHUNK_ID_FILLED_TH.unique())
    iterations = sum(df.groupby(['CHUNK_ID_FILLED_TH'])['ITERATION'].nunique())
    stats = pd.DataFrame({
        'CHUNKS':       [chunks],
        'ITERATIONS':   [iterations]
        })
    return stats

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up
def aggregate_to_confusion_matrix_per_chunk(df):
    confusion_matrix_per_chunk = df.groupby(['CHUNK_ID_FILLED_TH', 'ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum').reset_index()
    return confusion_matrix_per_chunk

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up
def aggregate_to_confusion_matrix_per_model(df):
    confusion_matrix_per_model = df.groupby(['ACCURACY_TYPE'])[['TP', 'FN', 'FP', 'TN']].agg('sum')
    return confusion_matrix_per_model

# Calculate statistical measures for evaluation of the model
def calc_metrics(df):
    TP_high = df.iloc[0,0]
    FN_high = df.iloc[0,1]
    FP_high = df.iloc[0,2]
    TN_high = df.iloc[0,3]

    TP_low = df.iloc[1,0]
    FN_low = df.iloc[1,1]
    FP_low = df.iloc[1,2]
    TN_low = df.iloc[1,3]

    fpr_high = FP_high/(FP_high+TN_high)
    tpr_high = TP_high/(TP_high+FN_high)
    fnr_high = FN_high/(TP_high+FN_high)
    tnr_high = TN_high/(FP_high+TN_high)
    acc_high = (TP_high+TN_high)/(TP_high+FN_high+FP_high+TN_high)
    f1s_high = TP_high/(TP_high+0.5*(FP_high+FN_high))

    fpr_low = FP_low/(FP_low+TN_low)
    tpr_low = TP_low/(TP_low+FN_low)
    fnr_low = FN_low/(TP_low+FN_low)
    tnr_low = TN_low/(FP_low+TN_low)
    acc_low = (TP_low+TN_low)/(TP_low+FN_low+FP_low+TN_low)
    f1s_low = TP_low/(TP_low+0.5*(FP_low+FN_low))

    metrics = pd.DataFrame({
        'ALARMTYPE':    ['High',        'Low'],
        'FPR':          [fpr_high,      fpr_low],
        'TPR':          [tpr_high,      tpr_low],
        'FNR':          [fnr_high,      fnr_low],
        'TNR':          [tnr_high,      tnr_low],
        'ACC':          [acc_high,      acc_low],
        'F1S':          [f1s_high,      f1s_low]
        }).round(4)
    
    return metrics


In [ ]:
# 01 - ARIMA - Steady Train Size 12

# Transform nested dictionary into data frame

confusion_matrices_per_chunk_01_arima_steady_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_12_hr_first1000)

confusion_matrices_per_chunk_01_arima_steady_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_12_bp_first1000)

confusion_matrices_per_chunk_01_arima_steady_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_12_o2_first1000)

# Calculate statistics about the number of chunks as well as chunk iteration combinations

stats_01_arima_steady_12_hr_first1000 = calc_stats(confusion_matrices_per_chunk_01_arima_steady_12_hr_first1000)

stats_01_arima_steady_12_bp_first1000 = calc_stats(confusion_matrices_per_chunk_01_arima_steady_12_bp_first1000)

stats_01_arima_steady_12_o2_first1000 = calc_stats(confusion_matrices_per_chunk_01_arima_steady_12_o2_first1000)

In [ ]:
# 02 - ARIMA - Expanding Train Size 12
# Transform nested dictionary into data frame

confusion_matrices_per_chunk_02_arima_expanding_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_expanding_12_hr_first1000)

confusion_matrices_per_chunk_02_arima_expanding_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_expanding_12_bp_first1000)

confusion_matrices_per_chunk_02_arima_expanding_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arima_expanding_12_o2_first1000)

# Calculate statistics about the number of chunks as well as chunk iteration combinations

stats_02_arima_expanding_12_hr_first1000 = calc_stats(confusion_matrices_per_chunk_02_arima_expanding_12_hr_first1000)

stats_02_arima_expanding_12_bp_first1000 = calc_stats(confusion_matrices_per_chunk_02_arima_expanding_12_bp_first1000)

stats_02_arima_expanding_12_o2_first1000 = calc_stats(confusion_matrices_per_chunk_02_arima_expanding_12_o2_first1000)

In [ ]:
# 03 - ARIMAX - Steady Train Size 12
# Transform nested dictionary into data frame

confusion_matrices_per_chunk_03_arimax_steady_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_12_hr_first1000)

confusion_matrices_per_chunk_03_arimax_steady_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_12_bp_first1000)

confusion_matrices_per_chunk_03_arimax_steady_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_12_o2_first1000)

# Calculate statistics about the number of chunks as well as chunk iteration combinations

stats_03_arimax_steady_12_hr_first1000 = calc_stats(confusion_matrices_per_chunk_03_arimax_steady_12_hr_first1000)

stats_03_arimax_steady_12_bp_first1000 = calc_stats(confusion_matrices_per_chunk_03_arimax_steady_12_bp_first1000)

stats_03_arimax_steady_12_o2_first1000 = calc_stats(confusion_matrices_per_chunk_03_arimax_steady_12_o2_first1000)

In [ ]:
# 04 - ARIMAX - Expanding Train Size 12
# Transform nested dictionary into data frame

confusion_matrices_per_chunk_04_arimax_expanding_12_hr_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_expanding_12_hr_first1000)

confusion_matrices_per_chunk_04_arimax_expanding_12_bp_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_expanding_12_bp_first1000)

confusion_matrices_per_chunk_04_arimax_expanding_12_o2_first1000 = dict_to_df(accuracy_dict_for_chunk_iterations_arimax_expanding_12_o2_first1000)

# Calculate statistics about the number of chunks as well as chunk iteration combinations

stats_04_arimax_expanding_12_hr_first1000 = calc_stats(confusion_matrices_per_chunk_04_arimax_expanding_12_hr_first1000)

stats_04_arimax_expanding_12_bp_first1000 = calc_stats(confusion_matrices_per_chunk_04_arimax_expanding_12_bp_first1000)

stats_04_arimax_expanding_12_o2_first1000 = calc_stats(confusion_matrices_per_chunk_04_arimax_expanding_12_o2_first1000)

## Analyze Data

In [ ]:
# 01 - ARIMA - Steady TRAIN Size 12

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up

confusion_matrix_per_chunk_01_arima_steady_12_hr_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_01_arima_steady_12_hr_first1000)

confusion_matrix_per_chunk_01_arima_steady_12_bp_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_01_arima_steady_12_bp_first1000)

confusion_matrix_per_chunk_01_arima_steady_12_o2_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_01_arima_steady_12_o2_first1000)

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up

confusion_matrix_01_arima_steady_12_hr_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_01_arima_steady_12_hr_first1000)

confusion_matrix_01_arima_steady_12_bp_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_01_arima_steady_12_bp_first1000)

confusion_matrix_01_arima_steady_12_o2_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_01_arima_steady_12_o2_first1000)

# Calculate statistical measures for evaluation of the model

metrics_01_arima_steady_12_hr_first1000 = calc_metrics(confusion_matrix_01_arima_steady_12_hr_first1000)

metrics_01_arima_steady_12_bp_first1000 = calc_metrics(confusion_matrix_01_arima_steady_12_bp_first1000)

metrics_01_arima_steady_12_o2_first1000 = calc_metrics(confusion_matrix_01_arima_steady_12_o2_first1000)

# Create overview table

model_01_arima_steady_12 = pd.DataFrame({
    'ID':           ['01',          '01'],
    'MODEL':        ['ARIMA',       'ARIMA'],
    'RESAMPLING':   ['Median',      'Median'],
    'EXOGENOUS':    ['n/a',         'n/a'],
    'TRAINSIZE':    ['Steady 12',   'Steady 12']
    })

model_01_arima_steady_12_hr_first1000 = pd.DataFrame({'PARAMETER': ['HR']})
overview_01_arima_steady_12_hr_first1000 = pd.concat([
    model_01_arima_steady_12_hr_first1000,
    model_01_arima_steady_12, 
    metrics_01_arima_steady_12_hr_first1000,
    stats_01_arima_steady_12_hr_first1000], axis=1).pad()

model_01_arima_steady_12_bp_first1000 = pd.DataFrame({'PARAMETER': ['BP']})
overview_01_arima_steady_12_bp_first1000 = pd.concat([
    model_01_arima_steady_12_bp_first1000,
    model_01_arima_steady_12, 
    metrics_01_arima_steady_12_bp_first1000,
    stats_01_arima_steady_12_bp_first1000], axis=1).pad()

model_01_arima_steady_12_o2_first1000 = pd.DataFrame({'PARAMETER': ['O2']})
overview_01_arima_steady_12_o2_first1000 = pd.concat([
    model_01_arima_steady_12_o2_first1000,
    model_01_arima_steady_12, 
    metrics_01_arima_steady_12_o2_first1000,
    stats_01_arima_steady_12_o2_first1000], axis=1).pad()

In [ ]:
# 02 - ARIMA - Expanding TRAIN Size

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up

confusion_matrix_per_chunk_02_arima_expanding_12_hr_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_02_arima_expanding_12_hr_first1000)

confusion_matrix_per_chunk_02_arima_expanding_12_bp_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_02_arima_expanding_12_bp_first1000)

confusion_matrix_per_chunk_02_arima_expanding_12_o2_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_02_arima_expanding_12_o2_first1000)

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up

confusion_matrix_02_arima_expanding_12_hr_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_02_arima_expanding_12_hr_first1000)

confusion_matrix_02_arima_expanding_12_bp_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_02_arima_expanding_12_bp_first1000)

confusion_matrix_02_arima_expanding_12_o2_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_02_arima_expanding_12_o2_first1000)

# Calculate statistical measures for evaluation of the model

metrics_02_arima_expanding_12_hr_first1000 = calc_metrics(confusion_matrix_02_arima_expanding_12_hr_first1000)

metrics_02_arima_expanding_12_bp_first1000 = calc_metrics(confusion_matrix_02_arima_expanding_12_bp_first1000)

metrics_02_arima_expanding_12_o2_first1000 = calc_metrics(confusion_matrix_02_arima_expanding_12_o2_first1000)

# Create overview table

model_02_arima_expanding_12 = pd.DataFrame({
    'ID':           ['02',              '02'],
    'MODEL':        ['ARIMA',           'ARIMA'],
    'RESAMPLING':   ['Median',          'Median'],
    'EXOGENOUS':    ['n/a',             'n/a'],
    'TRAINSIZE':    ['Expanding 12',    'Expanding 12']
    })

model_02_arima_expanding_12_hr_first1000 = pd.DataFrame({'PARAMETER': ['HR']})
overview_02_arima_expanding_12_hr_first1000 = pd.concat([
    model_02_arima_expanding_12_hr_first1000,
    model_02_arima_expanding_12, 
    metrics_02_arima_expanding_12_hr_first1000,
    stats_02_arima_expanding_12_hr_first1000], axis=1).pad()

model_02_arima_expanding_12_bp_first1000 = pd.DataFrame({'PARAMETER': ['BP']})
overview_02_arima_expanding_12_bp_first1000 = pd.concat([
    model_02_arima_expanding_12_bp_first1000,
    model_02_arima_expanding_12, 
    metrics_02_arima_expanding_12_bp_first1000,
    stats_02_arima_expanding_12_bp_first1000], axis=1).pad()

model_02_arima_expanding_12_o2_first1000 = pd.DataFrame({'PARAMETER': ['O2']})
overview_02_arima_expanding_12_o2_first1000 = pd.concat([
    model_02_arima_expanding_12_o2_first1000,
    model_02_arima_expanding_12, 
    metrics_02_arima_expanding_12_o2_first1000,
    stats_02_arima_expanding_12_o2_first1000], axis=1).pad()

In [ ]:
# 03 - ARIMAX - Steady TRAIN Size 12

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up

confusion_matrix_per_chunk_03_arimax_steady_12_hr_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_03_arimax_steady_12_hr_first1000)

confusion_matrix_per_chunk_03_arimax_steady_12_bp_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_03_arimax_steady_12_bp_first1000)

confusion_matrix_per_chunk_03_arimax_steady_12_o2_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_03_arimax_steady_12_o2_first1000)

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up

confusion_matrix_03_arimax_steady_12_hr_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_03_arimax_steady_12_hr_first1000)

confusion_matrix_03_arimax_steady_12_bp_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_03_arimax_steady_12_bp_first1000)

confusion_matrix_03_arimax_steady_12_o2_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_03_arimax_steady_12_o2_first1000)

# Calculate statistical measures for evaluation of the model

metrics_03_arimax_steady_12_hr_first1000 = calc_metrics(confusion_matrix_03_arimax_steady_12_hr_first1000)

metrics_03_arimax_steady_12_bp_first1000 = calc_metrics(confusion_matrix_03_arimax_steady_12_bp_first1000)

metrics_03_arimax_steady_12_o2_first1000 = calc_metrics(confusion_matrix_03_arimax_steady_12_o2_first1000)

# Create overview table

model_03_arimax_steady_12 = pd.DataFrame({
    'ID':           ['03',          '03'],
    'MODEL':        ['ARIMAX',      'ARIMAX'],
    'RESAMPLING':   ['Max',         'Min'],
    'EXOGENOUS':    ['Median',      'Median'],
    'TRAINSIZE':    ['Steady 12',   'Steady 12']
    })

model_03_arimax_steady_12_hr_first1000 = pd.DataFrame({'PARAMETER': ['HR']})
overview_03_arimax_steady_12_hr_first1000 = pd.concat([
    model_03_arimax_steady_12_hr_first1000,
    model_03_arimax_steady_12, 
    metrics_03_arimax_steady_12_hr_first1000,
    stats_03_arimax_steady_12_hr_first1000], axis=1).pad()

model_03_arimax_steady_12_bp_first1000 = pd.DataFrame({'PARAMETER': ['BP']})
overview_03_arimax_steady_12_bp_first1000 = pd.concat([
    model_03_arimax_steady_12_bp_first1000,
    model_03_arimax_steady_12, 
    metrics_03_arimax_steady_12_bp_first1000,
    stats_03_arimax_steady_12_bp_first1000], axis=1).pad()

model_03_arimax_steady_12_o2_first1000 = pd.DataFrame({'PARAMETER': ['O2']})
overview_03_arimax_steady_12_o2_first1000 = pd.concat([
    model_03_arimax_steady_12_o2_first1000,
    model_03_arimax_steady_12, 
    metrics_03_arimax_steady_12_o2_first1000,
    stats_03_arimax_steady_12_o2_first1000], axis=1).pad()

In [ ]:
# 04 - ARIMAX - Expanding TRAIN Size

# Create confusion matrix per chunk by aggregating so that TP, FN, FP, and TN of the several chunk iterations are summed up

confusion_matrix_per_chunk_04_arimax_expanding_12_hr_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_04_arimax_expanding_12_hr_first1000)

confusion_matrix_per_chunk_04_arimax_expanding_12_bp_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_04_arimax_expanding_12_bp_first1000)

confusion_matrix_per_chunk_04_arimax_expanding_12_o2_first1000 = aggregate_to_confusion_matrix_per_chunk(confusion_matrices_per_chunk_04_arimax_expanding_12_o2_first1000)

# Create confusion matrix by aggregating so that TP, FN, FP, and TN of all chunks are summed up

confusion_matrix_04_arimax_expanding_12_hr_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_04_arimax_expanding_12_hr_first1000)

confusion_matrix_04_arimax_expanding_12_bp_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_04_arimax_expanding_12_bp_first1000)

confusion_matrix_04_arimax_expanding_12_o2_first1000 = aggregate_to_confusion_matrix_per_model(confusion_matrices_per_chunk_04_arimax_expanding_12_o2_first1000)

# Calculate statistical measures for evaluation of the model

metrics_04_arimax_expanding_12_hr_first1000 = calc_metrics(confusion_matrix_04_arimax_expanding_12_hr_first1000)

metrics_04_arimax_expanding_12_bp_first1000 = calc_metrics(confusion_matrix_04_arimax_expanding_12_bp_first1000)

metrics_04_arimax_expanding_12_o2_first1000 = calc_metrics(confusion_matrix_04_arimax_expanding_12_o2_first1000)

# Create overview table

model_04_arimax_expanding_12 = pd.DataFrame({
    'ID':           ['04',              '04'],
    'MODEL':        ['ARIMAX',          'ARIMAX'],
    'RESAMPLING':   ['Max',             'Min'],
    'EXOGENOUS':    ['Median',          'Median'],
    'TRAINSIZE':    ['Expanding 12',    'Expanding 12']
    })

model_04_arimax_expanding_12_hr_first1000 = pd.DataFrame({'PARAMETER': ['HR']})
overview_04_arimax_expanding_12_hr_first1000 = pd.concat([
    model_04_arimax_expanding_12_hr_first1000,
    model_04_arimax_expanding_12, 
    metrics_04_arimax_expanding_12_hr_first1000,
    stats_04_arimax_expanding_12_hr_first1000], axis=1).pad()

model_04_arimax_expanding_12_bp_first1000 = pd.DataFrame({'PARAMETER': ['BP']})
overview_04_arimax_expanding_12_bp_first1000 = pd.concat([
    model_04_arimax_expanding_12_bp_first1000,
    model_04_arimax_expanding_12, 
    metrics_04_arimax_expanding_12_bp_first1000,
    stats_04_arimax_expanding_12_bp_first1000], axis=1).pad()

model_04_arimax_expanding_12_o2_first1000 = pd.DataFrame({'PARAMETER': ['O2']})
overview_04_arimax_expanding_12_o2_first1000 = pd.concat([
    model_04_arimax_expanding_12_o2_first1000,
    model_04_arimax_expanding_12, 
    metrics_04_arimax_expanding_12_o2_first1000,
    stats_04_arimax_expanding_12_o2_first1000], axis=1).pad()

In [20]:
# Create overview tables per parameter

overview_hr_first1000 = pd.concat([
    overview_01_arima_steady_12_hr_first1000,
    overview_02_arima_expanding_12_hr_first1000,
    overview_03_arimax_steady_12_hr_first1000,
    overview_04_arimax_expanding_12_hr_first1000], axis=0).reset_index(drop=True)

overview_bp_first1000 = pd.concat([
    overview_01_arima_steady_12_bp_first1000,
    overview_02_arima_expanding_12_bp_first1000,
    overview_03_arimax_steady_12_bp_first1000,
    overview_04_arimax_expanding_12_bp_first1000], axis=0).reset_index(drop=True)

overview_o2_first1000 = pd.concat([
    overview_01_arima_steady_12_o2_first1000,
    overview_02_arima_expanding_12_o2_first1000,
    overview_03_arimax_steady_12_o2_first1000,
    overview_04_arimax_expanding_12_o2_first1000], axis=0).reset_index(drop=True)

In [18]:
display(overview_hr_first1000)

,PARAMETER,ID,MODEL,RESAMPLING,EXOGENOUS,TRAINSIZE,ALARMTYPE,FPR,TPR,FNR,TNR,ACC,F1S,CHUNKS,ITERATIONS
0,HR,01,ARIMA,Median,n/a,Steady 12,High,0.0063,0.3541,0.6459,0.9937,0.9842,0.3982,671.0,25111.0
1,HR,01,ARIMA,Median,n/a,Steady 12,Low,0.0088,0.4873,0.5127,0.9912,0.9872,0.3743,671.0,25111.0
2,HR,02,ARIMA,Median,n/a,Expanding 12,High,0.0045,0.3811,0.6189,0.9955,0.9865,0.4534,671.0,25111.0
3,HR,02,ARIMA,Median,n/a,Expanding 12,Low,0.0035,0.4822,0.5178,0.9965,0.9924,0.5000,671.0,25111.0
4,HR,03,ARIMAX,Max,Median,Steady 12,High,0.0019,0.9011,0.0989,0.9981,0.9964,0.8970,671.0,25112.0
5,HR,03,ARIMAX,Min,Median,Steady 12,Low,0.0010,0.9372,0.0628,0.9990,0.9985,0.9108,671.0,25112.0
6,HR,04,ARIMAX,Max,Median,Expanding 12,High,0.0034,0.8966,0.1034,0.9966,0.9949,0.8581,671.0,25112.0
7,HR,04,ARIMAX,Min,Median,Expanding 12,Low,0.0014,0.9324,0.0676,0.9986,0.9980,0.8853,671.0,25112.0


In [22]:
display(overview_bp_first1000)

,PARAMETER,ID,MODEL,RESAMPLING,EXOGENOUS,TRAINSIZE,ALARMTYPE,FPR,TPR,FNR,TNR,ACC,F1S,CHUNKS,ITERATIONS
0,BP,01,ARIMA,Median,n/a,Steady 12,High,0.0231,0.3545,0.6455,0.9769,0.9485,0.3861,207.0,2407.0
1,BP,01,ARIMA,Median,n/a,Steady 12,Low,0.0111,0.2603,0.7397,0.9889,0.9668,0.3220,207.0,2407.0
2,BP,02,ARIMA,Median,n/a,Expanding 12,High,0.0196,0.3545,0.6455,0.9804,0.9518,0.4021,207.0,2406.0
3,BP,02,ARIMA,Median,n/a,Expanding 12,Low,0.0099,0.1781,0.8219,0.9901,0.9655,0.2385,207.0,2406.0
4,BP,03,ARIMAX,Max,Median,Steady 12,High,0.0044,0.9310,0.0690,0.9956,0.9925,0.9231,207.0,2407.0
5,BP,03,ARIMAX,Min,Median,Steady 12,Low,0.0052,0.9487,0.0513,0.9948,0.9934,0.9024,207.0,2407.0
6,BP,04,ARIMAX,Max,Median,Expanding 12,High,0.0074,0.9310,0.0690,0.9926,0.9896,0.8963,207.0,2407.0
7,BP,04,ARIMAX,Min,Median,Expanding 12,Low,0.0052,0.9487,0.0513,0.9948,0.9934,0.9024,207.0,2407.0


In [21]:
display(overview_o2_first1000)

,PARAMETER,ID,MODEL,RESAMPLING,EXOGENOUS,TRAINSIZE,ALARMTYPE,FPR,TPR,FNR,TNR,ACC,F1S,CHUNKS,ITERATIONS
0,O2,01,ARIMA,Median,n/a,Steady 12,High,0.0087,1.0000,0.0000,0.9913,0.9913,0.2609,438.0,11715.0
1,O2,01,ARIMA,Median,n/a,Steady 12,Low,0.0388,0.2851,0.7149,0.9612,0.9480,0.1759,438.0,11715.0
2,O2,02,ARIMA,Median,n/a,Expanding 12,High,0.0074,1.0000,0.0000,0.9926,0.9927,0.2951,438.0,11716.0
3,O2,02,ARIMA,Median,n/a,Expanding 12,Low,0.0171,0.2632,0.7368,0.9829,0.9689,0.2479,438.0,11716.0
4,O2,03,ARIMAX,Max,Median,Steady 12,High,0.0303,1.0000,0.0000,0.9697,0.9698,0.0923,438.0,11717.0
5,O2,03,ARIMAX,Min,Median,Steady 12,Low,0.0073,0.8893,0.1107,0.9927,0.9903,0.8087,438.0,11717.0
6,O2,04,ARIMAX,Max,Median,Expanding 12,High,0.0652,1.0000,0.0000,0.9348,0.9349,0.0451,438.0,11717.0
7,O2,04,ARIMAX,Min,Median,Expanding 12,Low,0.0123,0.9004,0.0996,0.9877,0.9857,0.7439,438.0,11717.0


In [24]:
overview_first1000 = pd.concat([
    overview_hr_first1000,
    overview_bp_first1000,
    overview_o2_first1000], axis=0).reset_index(drop=True)

display(overview_first1000)

,PARAMETER,ID,MODEL,RESAMPLING,EXOGENOUS,TRAINSIZE,ALARMTYPE,FPR,TPR,FNR,TNR,ACC,F1S,CHUNKS,ITERATIONS
0,HR,01,ARIMA,Median,n/a,Steady 12,High,0.0063,0.3541,0.6459,0.9937,0.9842,0.3982,671.0,25111.0
1,HR,01,ARIMA,Median,n/a,Steady 12,Low,0.0088,0.4873,0.5127,0.9912,0.9872,0.3743,671.0,25111.0
2,HR,02,ARIMA,Median,n/a,Expanding 12,High,0.0045,0.3811,0.6189,0.9955,0.9865,0.4534,671.0,25111.0
3,HR,02,ARIMA,Median,n/a,Expanding 12,Low,0.0035,0.4822,0.5178,0.9965,0.9924,0.5000,671.0,25111.0
4,HR,03,ARIMAX,Max,Median,Steady 12,High,0.0019,0.9011,0.0989,0.9981,0.9964,0.8970,671.0,25112.0
5,HR,03,ARIMAX,Min,Median,Steady 12,Low,0.0010,0.9372,0.0628,0.9990,0.9985,0.9108,671.0,25112.0
6,HR,04,ARIMAX,Max,Median,Expanding 12,High,0.0034,0.8966,0.1034,0.9966,0.9949,0.8581,671.0,25112.0
7,HR,04,ARIMAX,Min,Median,Expanding 12,Low,0.0014,0.9324,0.0676,0.9986,0.9980,0.8853,671.0,25112.0
8,BP,01,ARIMA,Median,n/a,Steady 12,High,0.0231,0.3545,0.6455,0.9769,0.9485,0.3861,207.0,2407.0
9,BP,01,ARIMA,Median,n/a,Steady 12,Low,0.0111,0.2603,0.7397,0.9889,0.9668,0.3220,207.0,2407.0


In [25]:
import pandas as pd
import pyarrow as pa

overview_first1000.to_parquet(path_to_data+'arima-x_forecast_overview_first1000.parquet', engine='pyarrow')

In [3]:
import pandas as pd
import pyarrow as pa

overview_first1000 = pd.read_parquet(path_to_data+'arima-x_forecast_overview_first1000.parquet', engine='pyarrow')

## Comparison for selected Chunk

In [ ]:
selected_chunkid = '200347.0_220045.0_2116-06-05 15:02:00'
selected_iteration = 11

In [ ]:
# 02 - ARIMA - Expanding TRAIN Size
# Confusion matrix for selected chunk
confusion_matrix_per_chunk_02_arima_expanding_df[
    confusion_matrix_per_chunk_02_arima_expanding_df.CHUNK_ID_FILLED_TH == selected_chunkid]

In [ ]:
# 02 - ARIMA - Expanding TRAIN Size
# Confusion matrix for selected iteration for selected chunk
confusion_matrices_per_chunk_02_arima_expanding_df[
    (confusion_matrices_per_chunk_02_arima_expanding_df.CHUNK_ID_FILLED_TH == selected_chunkid) &
    (confusion_matrices_per_chunk_02_arima_expanding_df.ITERATION == selected_iteration)
    ]

In [ ]:
# 03 - ARIMAX - Steady TRAIN Size 12
# Confusion matrix for selected chunk
confusion_matrix_per_chunk_03_arimax_steady_12_df[
    confusion_matrix_per_chunk_03_arimax_steady_12_df.CHUNK_ID_FILLED_TH == selected_chunkid]

In [ ]:
# 03 - ARIMAX - Steady TRAIN Size 12
# Confusion matrix for selected iteration for selected chunk
confusion_matrices_per_chunk_03_arimax_steady_12_df[
    (confusion_matrices_per_chunk_03_arimax_steady_12_df.CHUNK_ID_FILLED_TH == selected_chunkid) &
    (confusion_matrices_per_chunk_03_arimax_steady_12_df.ITERATION == selected_iteration)
    ]